In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.utils import np_utils
from pandas import Series , DataFrame, Panel
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.callbacks import EarlyStopping
from keras import metrics
from sklearn.decomposition import PCA

In [20]:
train = pd.read_csv('./Digit_Recognizer/train.csv')
TestX = pd.read_csv('./Digit_Recognizer/test.csv')

In [21]:
labels = train.iloc[1:,0].values
TrainX = (train.iloc[1:,1:].values).astype(float)
TrainY = np_utils.to_categorical(labels)

In [22]:
#Convert from 0-255 to 0-1
TrainX /= 255
TrainX -= np.std(TrainX)

In [23]:
print(TrainX.shape)

(41999, 784)


In [24]:
#PCA reduces the dimensionality ofo the data to increase speed of training
# pca = PCA(n_components=256)
# pca.fit(TrainX)
# TrainX = pca.transform(TrainX)
TrainX= TrainX.reshape(TrainX.shape[0],28,28,1)

In [ ]:
#adding layers to model. Pooling is a compression technique.
np.random.seed(42)

model = Sequential()
model.add(Conv2D(64, kernel_size=(4,4),
                 activation='relu',
                 input_shape=(28,28,1)))
model.add(Dropout(0.20))
model.add(Conv2D(32, kernel_size=(4, 4),activation='relu',))
model.add(Dropout(0.20))
model.add(Conv2D(16, kernel_size=(4, 4),activation='relu',))
model.add(Dropout(0.20))
# model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(TrainY.shape[1],activation  = 'softmax'))
early_stopping_monitor = EarlyStopping(monitor="acc",min_delta=0.05,patience=2)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(TrainX,TrainY, epochs = 16, verbose = 2, batch_size =50)

Epoch 1/16
378s - loss: 0.1538 - acc: 0.9523
Epoch 2/16


In [ ]:
TestX /= 255
TestX -= np.std(TrainX)
TestX= TestX.reshape(TestX.shape[0],28,28,1)
Prediction = model.predict_classes(TestX , verbose = 2)

In [ ]:
pd.DataFrame({"ImageId": list(range(1,len(Prediction)+1)), "Label": Prediction}).to_csv("submissionC.csv", index=False, header=True)